# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [5]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate tasks effectively and ask the right questions to my team members in order to gather the necessary insights about AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Use statistical modeling and machine learning to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to start by gathering real-time market data for AAPL to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yaho

Final Answer: 
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various subscription-based services. As of December 18, 2024, Apple Inc.'s stock price closed at $248.05, with a market cap of $3.749 trillion. The company's recent performance shows a YTD return of 29.47% and a 1-year return of 26.17%. Key financial metrics include a trailing P/E of 41.69, profit margin of 23.97%, and EPS of 6.08. Additionally, Apple Inc. has a strong presence in the technology sector and consumer electronics industry.

> Finished chain.
 

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various subscription-based services. As of December 18, 2024, Apple Inc.'s stock price closed at $248.05, with a market cap of $3.749 trillion. The company's recent performance shows a Y

Thought: I need to ask the Trading Strategy Developer a question to gather more insights for refining the trading strategies for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "What potential market opportunities and threats for AAPL have you identified based on the insights provided by the Data Analyst?", "context": "Insights and alerts about significant market opportunities or threats for AAPL"}


> Entering new CrewAgentExecutor chain...
I need to start by reading the website content to gather insights and alerts about significant market opportunities or threats for AAPL.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Thought: 
It seems like there was an issue reading the website content. I should try searching the internet for insights and alerts about potential market opportunities or threats f

Final Answer: 

Title: The Apple Inc (AAPL) Company: A Short SWOT Analysis
Link: https://finance.yahoo.com/news/apple-inc-aapl-company-short-050429085.html
Snippet: Some competitors have the resources to provide products at little or no profit, posing a threat to Apple's market share and profitability.

Title: Apple SWOT 2024 | SWOT Analysis of Apple | Business Strategy Hub
Link: https://bstrategyhub.com/swot-analysis-of-apple-apple-swot/
Snippet: 1. Antitrust threat: Apple Under Scrutiny. Apple is facing antitrust accusations that could lead to legal trouble and damage its reputation.

Title: Decoding Apple Inc (AAPL): A Strategic SWOT Insight
Link: https://finance.yahoo.com/news/decoding-apple-inc-aapl-strategic-050115589.html
Snippet: Global economic conditions and aggressive competition pose persistent threats to Apple's market share and profitability.

Title: Apple SWOT Analysis (2024) - Business Model Analyst
Link: https://businessmodelanalyst.com/apple-swot-analysis/?srsltid=Afm

I need to gather insights from my team members to effectively evaluate the risks associated with the proposed trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Evaluate risks associated with trading strategies for AAPL", "context": "The most effective execution methods for trading AAPL in the current market conditions involve utilizing limit orders and stop orders. It's important to closely monitor market trends and news related to AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather information on the current market trends and news related to AAPL to evaluate the risks associated with trading strategies for this stock.

Action: Search the internet
Action Input: {"search_query": "AAPL stock market trends and news"} 


Search results: Title: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/
Snippet: Find the latest Apple Inc. (AAPL) stock quote

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)